# 02 - Feature Engineering & Data Cleaning

**Objectif** : Nettoyer les données et créer de nouvelles variables (features) pour améliorer le modèle.

---

## Programme du notebook :
1. Charger les données
2. **Nettoyage des données** (NaN, valeurs saturées, outliers)
3. Créer des features temporelles (mois, saison)
4. Créer de nouveaux indices spectraux (ratios)
5. Fonction réutilisable pour le pipeline complet
6. Préparer les données de submission

---

## Stratégie de nettoyage :

| Dataset | Stratégie | Raison |
|---------|-----------|--------|
| **Training** | Supprimer NaN + outliers | On a assez de données (~8000 lignes) |
| **Submission** | Imputer par médiane | On doit prédire les 200 lignes |

---
## 1. Imports et chargement

In [ ]:
# Imports de base
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Configuration des graphiques
plt.rcParams['figure.figsize'] = (10, 6)

# Pour ignorer les warnings
import warnings
warnings.filterwarnings('ignore')

# Accéder au dossier src/
import sys
sys.path.append('..')

from src.paths import (
    WATER_QUALITY_FILE, LANDSAT_FILE, TERRACLIMATE_FILE,
    SUBMISSION_TEMPLATE, LANDSAT_SUBMISSION_FILE, TERRACLIMATE_SUBMISSION_FILE
)
from src.data.load_data import load_all, load_submission
from src.config import TARGETS, ALL_FEATURES, LANDSAT_BANDS, LANDSAT_INDICES

print("Imports OK!")

In [ ]:
# Charger les données (SANS nettoyage pour voir l'état brut)
X, y, site_ids, df = load_all(
    str(WATER_QUALITY_FILE),
    str(LANDSAT_FILE),
    str(TERRACLIMATE_FILE),
    features=ALL_FEATURES,
    fill_na=False  # On ne remplit PAS les NaN pour l'instant
)

print(f"\nColonnes disponibles: {list(df.columns)}")

---
## 2. Nettoyage des données (Training)

### Étapes :
1. Supprimer les lignes avec valeurs manquantes (NaN)
2. Supprimer les valeurs saturées Landsat (65535)
3. Calculer les médianes pour l'imputation du submission

In [ ]:
# =============================================================================
# 2.1 État initial et nettoyage
# =============================================================================

print("=" * 60)
print("NETTOYAGE DES DONNÉES TRAINING")
print("=" * 60)

# Constante pour les valeurs saturées
SATURATION_VALUE = 65535

# État initial
print(f"\nÉtat initial: {len(df)} lignes")
n_nan = df[ALL_FEATURES].isna().any(axis=1).sum()
print(f"  - Lignes avec NaN: {n_nan} ({100*n_nan/len(df):.1f}%)")
saturated_mask = (df[LANDSAT_BANDS] == SATURATION_VALUE).any(axis=1)
n_saturated = saturated_mask.sum()
print(f"  - Lignes avec valeurs saturées (65535): {n_saturated}")

# Nettoyage
df_clean = df.copy()

# Étape 1: Supprimer les NaN
before = len(df_clean)
df_clean = df_clean.dropna(subset=ALL_FEATURES)
print(f"\n1. Suppression NaN: {before} -> {len(df_clean)} lignes")

# Étape 2: Supprimer les valeurs saturées
before = len(df_clean)
saturated_mask = (df_clean[LANDSAT_BANDS] == SATURATION_VALUE).any(axis=1)
df_clean = df_clean[~saturated_mask]
print(f"2. Suppression saturées: {before} -> {len(df_clean)} lignes")

# Résultat
print(f"\n" + "=" * 60)
print(f"RÉSULTAT: {len(df_clean)} lignes ({100*len(df_clean)/len(df):.1f}% conservé)")
print("=" * 60)

In [ ]:
# =============================================================================
# 2.2 Calculer les médianes pour l'imputation (submission)
# =============================================================================

# On calcule les médianes sur le dataset NETTOYÉ
# Ces médianes seront utilisées pour imputer les valeurs manquantes du submission

medians = df_clean[ALL_FEATURES].median()

print("Médianes calculées (pour imputation submission):")
print("-" * 50)
for feature in ALL_FEATURES:
    print(f"  {feature}: {medians[feature]:.2f}")

# On continue à travailler avec df_clean pour le reste du notebook
df = df_clean.copy()
print(f"\n--> On travaille maintenant avec df_clean ({len(df)} lignes)")

---
## 3. Features temporelles

**Pourquoi ?** La qualité de l'eau varie selon les saisons (pluie, température, etc.)

**Features retenues :**
- `day_of_year` : position dans l'année (1-365), capture la saisonnalité fine
- `season` : saison (4 catégories), plus simple pour le modèle

In [ ]:
# Extraire le jour de l'année (1-365)
df['day_of_year'] = df['Sample Date'].dt.dayofyear

# Créer une variable "saison" (hémisphère sud)
def get_season(date):
    month = date.month
    if month in [12, 1, 2]: return 'summer'
    elif month in [3, 4, 5]: return 'autumn'
    elif month in [6, 7, 8]: return 'winter'
    else: return 'spring'

df['season'] = df['Sample Date'].apply(get_season)

print("Features temporelles créées:")
print(f"  - day_of_year: {df['day_of_year'].min()} à {df['day_of_year'].max()}")
print(f"  - season: {df['season'].value_counts().to_dict()}")

---
## 4. Corrélation des features Landsat avec les targets

On a maintenant toutes les features Landsat disponibles (6 bandes + 4 indices).

In [ ]:
# Quelles features Landsat sont disponibles ?
print("Features Landsat disponibles :")
landsat_features = LANDSAT_BANDS + LANDSAT_INDICES
for f in landsat_features:
    if f in df.columns:
        print(f"  - {f} : OK")
    else:
        print(f"  - {f} : MANQUANTE")

In [ ]:
# Corrélation des features Landsat avec les targets
landsat_in_df = [f for f in landsat_features if f in df.columns]

corr = df[landsat_in_df + TARGETS].corr()
corr_subset = corr.loc[landsat_in_df, TARGETS]

plt.figure(figsize=(10, 8))
sns.heatmap(corr_subset, annot=True, cmap='coolwarm', center=0, fmt='.2f')
plt.title('Corrélation Features Landsat vs Targets')
plt.tight_layout()
plt.show()

---
## 5. Créer de nouveaux ratios spectraux

Les ratios entre bandes peuvent capturer des informations sur la turbidité, la chlorophylle, etc.

In [ ]:
# Créer des ratios spectraux
if 'nir' in df.columns and 'green' in df.columns:
    df['nir_green_ratio'] = df['nir'] / (df['green'] + 0.0001)
    print("Feature 'nir_green_ratio' créée!")

if 'swir16' in df.columns and 'swir22' in df.columns:
    df['swir_ratio'] = df['swir16'] / (df['swir22'] + 0.0001)
    print("Feature 'swir_ratio' créée!")

# Corrélation des nouvelles features avec les targets
new_features = ['day_of_year', 'nir_green_ratio', 'swir_ratio']
new_features = [f for f in new_features if f in df.columns]

if new_features:
    corr = df[new_features + TARGETS].corr()
    corr_subset = corr.loc[new_features, TARGETS]
    
    plt.figure(figsize=(10, 5))
    sns.heatmap(corr_subset, annot=True, cmap='coolwarm', center=0, fmt='.2f')
    plt.title('Corrélation Nouvelles Features vs Targets')
    plt.tight_layout()
    plt.show()

---
## 6. Résumé des features

In [11]:
def create_features(df):
    """
    Crée toutes les features supplémentaires.
    
    Features temporelles :
    - day_of_year : jour de l'année (1-365)
    - season : saison (summer, autumn, winter, spring)
    
    Features spectrales :
    - nir_green_ratio : ratio NIR/Green
    - swir_ratio : ratio SWIR16/SWIR22
    
    Paramètres:
        df : DataFrame avec les données brutes
    
    Retourne:
        DataFrame avec les nouvelles features
    """
    df = df.copy()
    
    # Features temporelles
    if 'Sample Date' in df.columns:
        # Jour de l'année (1-365)
        df['day_of_year'] = df['Sample Date'].dt.dayofyear
        
        # Saison (hémisphère sud)
        def get_season(date):
            month = date.month
            if month in [12, 1, 2]: return 'summer'
            elif month in [3, 4, 5]: return 'autumn'
            elif month in [6, 7, 8]: return 'winter'
            else: return 'spring'
        
        df['season'] = df['Sample Date'].apply(get_season)
    
    # Ratios spectraux
    if 'nir' in df.columns and 'green' in df.columns:
        df['nir_green_ratio'] = df['nir'] / (df['green'] + 0.0001)
    
    if 'swir16' in df.columns and 'swir22' in df.columns:
        df['swir_ratio'] = df['swir16'] / (df['swir22'] + 0.0001)
    
    return df

print("Fonction create_features() définie!")
print("\nFeatures créées par cette fonction :")
print("  - day_of_year (temporelle)")
print("  - season (temporelle)")
print("  - nir_green_ratio (spectrale)")
print("  - swir_ratio (spectrale)")

Fonction create_features() définie!

Features créées par cette fonction :
  - day_of_year (temporelle)
  - season (temporelle)
  - nir_green_ratio (spectrale)
  - swir_ratio (spectrale)


---
## 7. Préparation des données de Submission

On charge les données de submission et on impute les valeurs manquantes avec les médianes du training.

---
## 8. Résumé

### Données préparées :

| Dataset | Lignes | Nettoyage |
|---------|--------|-----------|
| **Training** (`df`) | ~7900 | NaN et saturées supprimés |
| **Submission** (`df_sub_imputed`) | 200 | NaN imputés par médiane |

### Features disponibles (15 au total) :

| Type | Features |
|------|----------|
| **Landsat bandes** (6) | blue, green, red, nir, swir16, swir22 |
| **Landsat indices** (4) | NDVI, NDWI, NDMI, MNDWI |
| **Climat** (1) | pet |
| **Temporelles** (2) | day_of_year, season |
| **Ratios** (2) | nir_green_ratio, swir_ratio |

### Prochaine étape :

**Notebook 03 : Modélisation** - Entraîner et évaluer des modèles

In [ ]:
# Imputer les valeurs manquantes avec les médianes du training
df_sub_imputed = df_sub.copy()

for feature in ALL_FEATURES:
    if df_sub_imputed[feature].isna().any():
        n_missing = df_sub_imputed[feature].isna().sum()
        df_sub_imputed[feature] = df_sub_imputed[feature].fillna(medians[feature])
        print(f"  {feature}: {n_missing} valeurs imputées avec médiane = {medians[feature]:.2f}")

# Créer les features supplémentaires sur submission
df_sub_imputed = create_features(df_sub_imputed)

print(f"\nDataset submission prêt: {len(df_sub_imputed)} lignes")
print(f"Valeurs manquantes restantes: {df_sub_imputed[ALL_FEATURES].isna().sum().sum()}")

In [ ]:
# Charger les données de submission
X_sub, df_sub = load_submission(
    str(SUBMISSION_TEMPLATE),
    str(LANDSAT_SUBMISSION_FILE),
    str(TERRACLIMATE_SUBMISSION_FILE),
    features=ALL_FEATURES,
    fill_na=False  # On impute manuellement avec nos médianes
)

print(f"\nValeurs manquantes avant imputation:")
print(df_sub[ALL_FEATURES].isna().sum())

---
## 9. Visualisation du DataFrame final

Aperçu des données nettoyées et des features créées.